In [1]:


import openai, inspect, os
print("Loaded from:", os.path.dirname(inspect.getfile(openai)))
import pkg_resources
print("Reported version:", pkg_resources.get_distribution("openai").version)

from openai import OpenAI
import os
from dotenv import load_dotenv
load_dotenv(override=True)
api_key=os.getenv("GEMINI_API_KEY")


Loaded from: c:\llmpractice\gemini-env\Lib\site-packages\openai
Reported version: 2.9.0


In [ ]:
user_prompt=""" you are give the data related to some website.
 as a helpfull assistant you need to extract the data and create a nice summarized report  of the company    using the data."""

In [ ]:
system_prompt="""  you are a helpful assistant that helps to summarize the about the company   from the provoided data and even shows the live data/news if provoided """

In [35]:
def webprompt(content):
    return [{"role":"user","content":user_prompt+content},{"role":"system","content":system_prompt}]



In [36]:
client=OpenAI(
    api_key=api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai",

   
)
import sys
print(sys.executable)


c:\llmpractice\gemini-env\Scripts\python.exe


In [14]:
!pip install beautifulsoup4


  Using cached beautifulsoup4-4.14.3-py3-none-any.whl (107 kB)
  Using cached soupsieve-2.8-py3-none-any.whl (36 kB)



[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
!pip install requests


     ---------------------------------------- 64.7/64.7 kB 1.2 MB/s eta 0:00:00
     -------------------------------------- 107.0/107.0 kB 1.5 MB/s eta 0:00:00
     ------------------------------------ 131.1/131.1 kB 857.0 kB/s eta 0:00:00



[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [37]:
from bs4 import BeautifulSoup
import requests


# Standard headers to fetch a website
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}


def webcontent(url):
    """
    Return the title and contents of the website at the given url;
    truncate to 2,000 characters as a sensible limit
    """
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    title = soup.title.string if soup.title else "No title found"
    if soup.body:
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        text = soup.body.get_text(separator="\n", strip=True)
    else:
        text = ""
    return (title + "\n\n" + text)[:2_000]


def fetch_website_links(url):
    """
    Return the links on the webiste at the given url
    I realize this is inefficient as we're parsing twice! This is to keep the code in the lab simple.
    Feel free to use a class and optimize it!
    """
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    links = [link.get("href") for link in soup.find_all("a")]
    return [link for link in links if link]


In [26]:
!pip install Markdown

     ------------------------------------ 107.7/107.7 kB 521.1 kB/s eta 0:00:00



[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [38]:
def summarize(url):
    content=webcontent(url)
    prompt=webprompt(content)
    response=client.chat.completions.create(model="gemini-2.5-flash",messages=prompt,temperature=0.5)
    return response.choices[0].message.content
    

In [39]:
from IPython.display import Markdown, display
display(Markdown(summarize("https://www.google.com")))

The data you've provided gives us a glimpse into Google's public-facing services, operational aspects, and regional presence, particularly in India. It's important to note that this data primarily describes the *features and interface* of Google's services, rather than traditional financial or performance metrics (like revenue, user growth, or market share).

Here's a summary report based on the provided information:

---

### **Google - Operational & Service Overview**

**Date of Report:** October 26, 2023 (Based on current processing date)
**Data Source:** Provided text snippet (appears to be from a Google search page footer/sidebar details and an upload interface message).

---

**1. Company Overview & Core Offerings:**
Google continues to operate as a leading provider of internet services, with its core offerings prominently featured:
*   **Search Engine:** The primary interface for information retrieval, supported by detailed "How Search works" information, "Advanced search" options, and user data management features ("Your data in Search," "Search history").
*   **Key Services:** Gmail (email), Google Images (image search), Google Store (e-commerce for hardware/software).
*   **Business & Advertising:** Dedicated sections for "Advertising" and "Business" indicate a strong focus on enterprise solutions and revenue generation through ad platforms.
*   **AI Integration:** The mention of "AI Mode" suggests ongoing integration of artificial intelligence into user functionalities, though specific details are not provided.

**2. User Interface & Experience Features:**
*   **Account Management:** Standard "Sign in" functionality is available.
*   **Content Uploads:** Users can upload images and files. Supported formats are explicitly listed: `.avif, .bmp, .jpeg, .pdf, .png, .webp`. An error message indicates strict adherence to these formats.
*   **Customization:** A "Dark theme" option is available for user interface preference.
*   **Feedback & Support:** Users can "Send feedback" and access "Search help," indicating channels for user engagement and issue resolution.
*   **Privacy & Terms:** Links to "Privacy" and "Terms" are prominently displayed, emphasizing transparency and legal compliance.

**3. Regional Focus & Localization (India Specific):**
Google demonstrates a significant commitment to localization, particularly in India:
*   **Geographic Presence:** Explicitly states "India."
*   **Language Support:** Services are offered in a wide array of Indian regional languages, facilitating broader accessibility and user engagement:
    *   हिन्दी (Hindi)
    *   বাংলা (Bengali)
    *   తెలుగు (Telugu)
    *   மராठी (Marathi)
    *   தமிழ் (Tamil)
    *   ગુજરાતી (Gujarati)
    *   ಕನ್ನಡ (Kannada)
    *   മലയാളം (Malayalam)
    *   ਪੰਜਾਬੀ (Punjabi)

**4. Operational Settings & Controls:**
*   **Search Settings:** Users have granular control over their search experience via "Settings" and "Search settings."
*   **Data Management:** Features like "Your data in Search" and "Search history" provide users with tools to manage their personal data and search activities.

---

### **Summary of Performance (Based on Provided Data):**

Based *solely* on the provided text, Google appears to be a **robust and user-centric platform** with a strong focus on:
*   **Comprehensive Service Offerings:** Covering search, communication, e-commerce, and business solutions.
*   **User Experience:** Providing customization options (Dark theme), clear upload guidelines, and feedback mechanisms.
*   **Global/Local Reach:** Demonstrating significant investment in localization, especially in the Indian market, by supporting numerous regional languages.
*   **Transparency & Control:** Offering users access to privacy policies, terms, and data management tools.

**Limitations:**
This report is based on a very limited dataset and does **not** include financial performance, market share, user growth metrics, competitive analysis, or recent strategic initiatives. It primarily reflects the functional and accessibility aspects of Google's public-facing web interface.

---

### **Live Data / News (Not Provided in Original Data):**

The data you provided does not contain any live news feeds or real-time performance metrics. To get live data or news about Google's performance, you would typically need to consult sources such as:

*   **Financial News Outlets:** Reuters, Bloomberg, Wall Street Journal, Financial Times.
*   **Tech News Sites:** TechCrunch, The Verge, CNET.
*   **Google's Investor Relations Website:** For official earnings reports, press releases, and SEC filings.
*   **Stock Market Data Providers:** For real-time stock prices and market capitalization (e.g., Google's parent company, Alphabet Inc. - GOOGL, GOOG).

**Example of what "Live Data/News" *could* look like if provided:**

*(If I had access to real-time news, I would insert something like this here, but it's not possible with the given input):*

*   **[Date/Time]:** Alphabet Inc. (Google's parent company) stock [GOOGL] is trading at [Current Price] USD, [Change] today.
*   **[Date/Time]:** Recent news indicates Google is investing heavily in AI infrastructure, with reports of new data centers being built to support its generative AI initiatives.
*   **[Date/Time]:** Google announced a new partnership with [Company Name] to enhance cloud services for [Industry].

---